# Facts: training


In [ ]:
#@title

!pip -q install shap

import os
from functools import reduce

import numpy as np
import pandas as pd
import tensorflow as tf
import shap
from sklearn.preprocessing import LabelEncoder, StandardScaler

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from google.colab import drive

sns.set(palette=sns.color_palette("hls", 8))
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

drive.mount('/content/drive')

In [ ]:
SEED = np.random.RandomState(180120342)

DATASET = '/content/drive/My Drive/datasets/facts.csv'

In [ ]:
x = pd.read_csv(DATASET)

x.head()

In [ ]:
xt = x.title.copy()
xt[pd.isnull(xt)] = ''
x['full_title_text'] = xt + ' ' + x.text

x.head()

In [ ]:
plt.figure(figsize=(10, 4))

plt.subplot(131)
sns.barplot(*np.unique(x.source.astype(str), return_counts=True))
plt.xticks(rotation=-45)

plt.subplot(132)
sns.barplot(*np.unique(x.stage, return_counts=True))

plt.subplot(133)
sns.barplot(*np.unique(x.target, return_counts=True))
plt.tight_layout()

plt.figure(figsize=(10, 4))
sns.barplot(*np.unique(x.category.astype(str), return_counts=True))
plt.xticks(rotation=90)
plt.tight_layout();

### Determining Categories

In [ ]:
x_train = x[x.stage == 'train'][['full_title_text', 'category']].dropna()

In [ ]:
x_train.head()

In [ ]:
from sklearn import metrics

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer

category_m = Pipeline([
    ('tfidf', TfidfVectorizer(strip_accents='unicode',
                              stop_words='english',
                              ngram_range=(1, 3),
                              max_features=4096,
                              lowercase=True)),
    ('rf', LinearRegression())
])

category_m.fit(x_train.full_title_text, x_train.category.astype('str'));

In [ ]:
e_train.shape